# Stock Data & Sentiment Analysis Agent with Agno

This is an example of Stock Data & Sentiment Analysis Agent. It is used to demonstrate the creation of AI Agents that are autonomously driven for specific tasks that can follow rules and adapt to input. In this case, we focus on tasks such as testing and iterating different tool combination including Google search + yfinance.

For this example we leverage:

- Agno, a modern agent framework that’s actually built for tools and performance. It is model-agnostic but in this example we leverage a multi-model open weight model provided on Scaleway cloud infrastructure. Also the framework supports custom Python tools, which means we can easily extend it in teh future with custom tools for stock data analysis for example.  
- yfinance: Used to fetch financial data from Yahoo Finance.
- Google Search: Used to search the web for news articles.

## Execution Variables Setup

This section includes all the necessary variables to the execution of the data pipeline and the agentic flow. We also perform a quick test against the model endpoint with a simple agno agent before proceeding.

In [17]:
# Load environment variables from .env file
import os
from dotenv import load_dotenv
load_dotenv()

# Environment variables for Google GenAI API inference
google_api_key = os.getenv("GOOGLE_API_KEY", "")
google_model = "gemini-2.5-flash"
google_model_eval = "gemini-2.5-pro"

# Environment variables for Scaleway GenAI API inference
scw_project_id = os.getenv("SCW_DEFAULT_PROJECT_ID", "")
scw_url = "https://api.scaleway.ai/" + scw_project_id + "/v1"
scw_api_key = os.getenv("SCW_SECRET_KEY", "")
scw_model = "deepseek-r1-distill-llama-70b"

In [2]:
from os import getenv
from agno.agent import Agent
from agno.models.openai.like import OpenAILike

agent = Agent(
    model=OpenAILike(
        id=scw_model,
        api_key=scw_api_key,
        base_url=scw_url,
    ),
    markdown=True,
    stream=True,
)

# Print the response in the terminal
agent.print_response("Share a 2 sentence horror story.")

Output()

In [18]:
# Sentiment Analysis Agent
from agno.tools.googlesearch import GoogleSearchTools
from agno.tools.yfinance import YFinanceTools
sentiment_agent = Agent(
    name="Sentiment Analysis Agent",
    role="Search and interpret news articles",
    model=OpenAILike(
        id=scw_model,
        api_key=scw_api_key,
        base_url=scw_url,
    ),
    tools=[
        GoogleSearchTools(),
        YFinanceTools(include_tools=["get_company_news"]),
    ],
    instructions=[
        "Find relevant news articles for each company and critically analyze the news sentiment.",
        "Provide sentiment scores from 1 (negative) to 10 (positive) with reasoning and sources."
        "Cite your sources. Be specific, crtical and provide relevant links."
    ],
    markdown=True,
    stream=True,
)

DEBUG Function: google_search registered with google_search_tools

DEBUG Function: get_company_news registered with yfinance_tools

In [19]:
# Financial Analyst Agent
basic_stock_agent = Agent(
    name="Basic Financial Data Agent",
    role="Retrieve basic company financial data and expertly interpret trends and data using a rigourously analytical approach",
    model=OpenAILike(
        id=scw_model,
        api_key=scw_api_key,
        base_url=scw_url,
    ),
    tools=[YFinanceTools(include_tools=["get_current_stock_price", 
                                        "get_historical_stock_prices",
                                        "get_company_info",
                                        "get_stock_fundamentals", 
                                        "get_analyst_recommendations"])],
    instructions=[
        "Retrieve stock prices, analyst recommendations, and key summary financial data.",
        "Focus on company funamentals and trends, presenting the data in tables with key insights."
    ],
    markdown=True,
    stream=True,
)

DEBUG Function: get_current_stock_price registered with yfinance_tools

DEBUG Function: get_company_info registered with yfinance_tools

DEBUG Function: get_stock_fundamentals registered with yfinance_tools

DEBUG Function: get_analyst_recommendations registered with yfinance_tools

DEBUG Function: get_historical_stock_prices registered with yfinance_tools

In [20]:
# Historical Data Analyst Agent
historical_financials_agent = Agent(
    name="Comprehensive Financial Data Agent",
    role="Retrieve comprehensive company financial data and expertly interpret trends and data using a rigourously analytical approach",
    model=OpenAILike(
        id=scw_model,
        api_key=scw_api_key,
        base_url=scw_url,
    ),
     tools=[YFinanceTools(include_tools=["get_company_info", 
                                        "get_key_financial_ratios",
                                        "get_income_statements", 
                                        "get_historical_stock_prices"])],
    instructions=[
        "Retrieve key financial ratios, income statement, balance sheet, cash flow statements and historical price data.",
        "Focus on company fundamentals and trends, presenting the data in tables with key insights."
    ],
    markdown=True,
    stream=True,
)

DEBUG Function: get_company_info registered with yfinance_tools

DEBUG Function: get_income_statements registered with yfinance_tools

DEBUG Function: get_key_financial_ratios registered with yfinance_tools

DEBUG Function: get_historical_stock_prices registered with yfinance_tools

In [21]:
# Data Synthesis / Executive Agent
executive_agent = Agent(
    name="Executive Agent",
    role="You are a seasoned portfolio manager and executive. You are tasked with making data-driven decisions and drawing conclusions through critical assessment",
    model=OpenAILike(
        id=scw_model,
        api_key=scw_api_key,
        base_url=scw_url,
    ),
    instructions=[
        "Check outputs for accuracy and completeness.",
        "Synthesize data to provide a final sentiment score (1-10) with justifcation."
        "Finally, after critical reasoning, provide a BUY, SELL or HOLD recommendation for the stock."
    ],
    markdown=True,
    stream=True,
)

## Create the Multi Agent Orchestration (Build Team)

These few lines of code set up a sophisticated multi agent system with specific roles that collaborate to analyze a stock.

The `agent_team` acts as a conductor, picking the right agent work flow and ensuring that each agent contributes its insights and that the final output is well-structured, data-driven, and easily understandable

In [22]:
# Full Team of Agents
from agno.team import Team

team = Team(
    model=OpenAILike(
        id=scw_model,
        api_key=scw_api_key,
        base_url=scw_url,
    ),
    members=[sentiment_agent, basic_stock_agent, historical_financials_agent, executive_agent],
    instructions=[
        "Combine the expertise of all the agents part of your team to provide a cohesive, highly analytical and well-supported response",
        "Always include references and dates for all data points and sources. Your context is current as of today."
        "Present data in structured tables for clarity, ensuring all conclusions are data-driven.",
        "Explain the methodlogy and always determine a BUY, SELL, or HOLD recommendation.",
    ],
    markdown=True,
    stream=True,
    debug_mode=True,   # Set to True to view the detailed logs and see the compiled system message
)

In [23]:
team.print_response(
    "Analyze the sentiment for the following companies for the month of December 2024 (your context is up-to-date as of Jan 16 2025): JPM, META. \n\n"
    "1. **Sentiment Analysis**: Search for relevant news articles and interpret the sentiment for each company. Provide the sentiment score"
    "2. **Financial Data**: Analyze stock price and company fundamental financial metrics and other resources. Highlight key trends"
    "3. **Consolidated Analysis**: Combine the insights from the various srouces such as sentiment analysis and financial data to assign a final BUY, SELL or HOLD recommendation"
    "Ensure your response is accurate, comprehensive, analytical and well-reasoned. Include references to sources with publication dates.",
    stream=True
)

DEBUG ***** Session ID: 85df2453-4485-4a82-9ba4-ebd8f0bfe3e8 *****

DEBUG ****** Team ID: 5a65c7aa-5bbc-46eb-8815-591a865a4a11 *******

DEBUG Creating new TeamSession: 85df2453-4485-4a82-9ba4-ebd8f0bfe3e8

DEBUG Processing tools for model

DEBUG Added tool delegate_task_to_member

...

DEBUG * Tokens per second:           24.9840 tokens/s

DEBUG ************************  METRICS  *************************

DEBUG ---------------- OpenAI Response Stream End ----------------

DEBUG Added RunOutput to Team Session

DEBUG **** Team Run End: 2e1f7d1e-a60b-4a0f-91d0-9e725d551385 ****